<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [1]:
%matplotlib inline
%config InlineBackend.figure_formats = {'png', 'retina'}

%load_ext autoreload
# "1" means: always reload modules marked with "%aimport"
# "2" means: always reload all modules except those marked with "%aimport"
%autoreload 1

# from pathlib import Path
# import sys
# import os

import matplotlib.pyplot as plt

import numpy as np
import pandas as pd

from tqdm import tqdm_notebook

from postal.expand import expand_address
# from postal.parser import parse_address

from fuzzywuzzy import process
from fuzzywuzzy import fuzz

plt.rc('figure', figsize=(8.0, 6.0))

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
# np.set_printoptions(precision=4, linewidth=100)

In [2]:
filename_inspect = 'data/DOHMH_New_York_City_Restaurant_Inspection_Results.csv'
filename_rest = 'data/restaurants_nyc_dedupe.csv'

df_rest = pd.read_csv(filename_rest)
df_inspect = pd.read_csv(filename_inspect)

In [3]:
df_rest['address'] = df_rest['address'].str.replace('.', '')
df_rest['address'] = df_rest['address'].str.replace(f', NY, ', ', New York, ', case=True, regex=False)

# df_rest['name'] = df_rest['name'].str.title()
# df_rest['address'] = df_rest['address'].str.title()

In [4]:
df_inspect = df_inspect.dropna(subset=['DBA', 'BUILDING', 'STREET', 'BORO', 'ZIPCODE'], how='any')
df_inspect['STATE'] = 'New York'

df_inspect['BUILDING'] = df_inspect['BUILDING'].astype(str)
df_inspect['ZIPCODE'] = df_inspect['ZIPCODE'].astype(int).astype(str)

df_inspect = df_inspect.rename(columns={'DBA': 'name'})

df_inspect['name'] = df_inspect['name'].str.title()
df_inspect['BORO'] = df_inspect['BORO'].str.title()
df_inspect['STREET'] = df_inspect['STREET'].str.title().str.replace('.', '')

In [5]:
# df_rest['address'].str.split(',', n=1).str[0].str.rsplit(' ').str[-1].value_counts().index.tolist()

to_replace = {
    'St': 'Street',
    'Ave': 'Avenue',
    'Ln': 'Lane',
    'Pl': 'Place',
    'Blvd': 'Boulevard',
    'Ste': 'Suite',
    'Cir': 'Circle',
    'Fl': 'Floor',
    'Plz': 'Plaza',
    'Tpke': 'Turnpike',
    'Rd': 'Road',
    'Frnt': 'Front',
    'Hwy': 'Highway',
    'Sq': 'Square',
    'Expy': 'Expressway',
    'Ctr': 'Center',
    'Ter': 'Terrace',
    'Dr': 'Drive',
    'Bld': 'Building',
}

for k, v in to_replace.items():
    df_rest['address'] = df_rest['address'].str.replace(rf'\b{k}\b', v, case=False, regex=True)
    df_inspect['STREET'] = df_inspect['STREET'].str.replace(rf'\b{k}\b', v, case=False, regex=True)

In [6]:
df_inspect['address'] = df_inspect['BUILDING'].str.cat(
    df_inspect['STREET'], sep=' ').str.cat(
    df_inspect['BORO'], sep=', ').str.cat(
    df_inspect['STATE'], sep=', ').str.cat(
    df_inspect['ZIPCODE'], sep=', ')

df_inspect['address'] = df_inspect['address'].str.replace('.', '')

In [7]:
df_rest.sample(n=5)

,address,name
9075,"201 Hempstead Turnpike, Elmont, New York, 11003",SUBWAY®
3988,"1070 2nd Avenue, New York, New York, 10022",NY Jumbo Bagels
1048,"2137 Nostrand Avenue, Brooklyn, New York, 11210",Fisherman's Cove
7539,"519 6th Avenue, New York, New York, 10011",Cafe Water
8548,"1005 2nd Avenue, New York, New York, 10022",Primavera Pizza


In [8]:
df_inspect.sample(n=5)

,CAMIS,name,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,VIOLATION CODE,VIOLATION DESCRIPTION,CRITICAL FLAG,SCORE,GRADE,GRADE DATE,RECORD DATE,INSPECTION TYPE,STATE,address
80031,41653607,Chop'T,Manhattan,18,East 23 Street,10010,6467557837,Salads,06/08/2015,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,13.0,A,06/08/2015,02/06/2019,Cycle Inspection / Re-inspection,New York,"18 East 23 Street, Manhattan, New York, 10010"
185458,50080484,Burgerology,Queens,3129,Ditmars Boulevard,11105,5164391009,American,12/26/2018,Violations were cited in the following area(s).,02I,Food prepared from ingredients at ambient temp...,Critical,20.0,B,12/26/2018,02/06/2019,Cycle Inspection / Re-inspection,New York,"3129 Ditmars Boulevard, Queens, New York, 11105"
374254,40859971,Hong Cheong,Manhattan,2393,Frederick Douglas Boulevard,10027,2123166688,Chinese,05/25/2017,Violations were cited in the following area(s).,10F,Non-food contact surface improperly constructe...,Not Critical,7.0,A,05/25/2017,02/06/2019,Cycle Inspection / Initial Inspection,New York,"2393 Frederick Douglas Boulevard, Manhattan, N..."
69653,41084481,Joe'S Ginger Restaurant,Manhattan,25,Pell Street,10013,2122850333,Chinese,07/26/2016,Violations were cited in the following area(s).,04L,Evidence of mice or live mice present in facil...,Critical,37.0,NaN,NaN,02/06/2019,Cycle Inspection / Initial Inspection,New York,"25 Pell Street, Manhattan, New York, 10013"
81333,50004389,Roxy Diner,Manhattan,694,8Th Avenue,10036,2125759700,American,04/13/2016,Violations were cited in the following area(s).,06E,"Sanitized equipment or utensil, including in-u...",Critical,10.0,NaN,NaN,02/06/2019,Cycle Inspection / Initial Inspection,New York,"694 8Th Avenue, Manhattan, New York, 10036"


In [9]:
# expand_address('11 W 32nd St Frnt 1 # 6, New York, NY, 10001')

# fuzz.ratio(row_rest['address_expand'][0], '31-91 21st st astoria new york 11106')

In [10]:
df_rest['address_expand'] = df_rest['address'].apply(expand_address)
df_inspect['address_expand'] = df_inspect['address'].apply(expand_address)

In [ ]:
options = df_inspect['address_expand'].apply(lambda x: x[0])

df_inspect['fuzzy_matched'] = False
df_rest['fuzzy_matched'] = False
df_rest['matched_address'] = np.nan
df_rest['matched_name'] = np.nan
df_rest['matched_index'] = np.nan

scorer = fuzz.ratio
score_cutoff = 95

for idx_rest, row_rest in tqdm_notebook(df_rest.iterrows(), total=df_rest.shape[0]):
    result = process.extractOne(row_rest['address_expand'][0], options,
                                scorer=scorer,
                                score_cutoff=score_cutoff)
    if result:
        df_inspect.loc[idx_insp, 'fuzzy_matched'] = True
        df_rest.loc[idx_rest, 'fuzzy_matched'] = True
        df_rest.loc[idx_rest, 'matched_address'] = df_inspect.loc[result[2], 'address']
        df_rest.loc[idx_rest, 'matched_name'] = df_inspect.loc[result[2], 'name']
        df_rest.loc[idx_rest, 'matched_index'] = result[2]
        # print('=' * 30)
        # print(result)
        
        # remove this entry from consideration
        options = options[~options.index.isin([result[2]])]


In [ ]:
# df_inspect['fuzzy_matched'] = False
# df_rest['fuzzy_matched'] = False
# df_rest['matched_address'] = np.nan
# df_rest['matched_name'] = np.nan
# df_rest['matched_index'] = np.nan

# scorer = fuzz.ratio
# score_cutoff = 95

# for idx_rest, row_rest in tqdm_notebook(df_rest.iterrows(), total=df_rest.shape[0]):
#     for idx_insp, row_insp in df_inspect.loc[~df_inspect['fuzzy_matched']].iterrows():
#         # compare to 0th address in df_inspect
#         if scorer(row_rest['address_expand'][0], row_insp['address_expand'][0]) >= score_cutoff:
#             df_inspect.loc[idx_insp, 'fuzzy_matched'] = True
#             df_rest.loc[idx_rest, 'fuzzy_matched'] = True
#             df_rest.loc[idx_rest, 'matched_address'] = row_insp['address']
#             df_rest.loc[idx_rest, 'matched_name'] = row_insp['name']
#             df_rest.loc[idx_rest, 'matched_index'] = idx_insp
#             # print('=' * 30)
#             # print(idx_insp)
#             # print(result)
#             break

# #         # search through all addresses in df_inspect
# #         result = process.extractOne(row_rest['address_expand'][0], row_insp['address_expand'],
# #                                     scorer=scorer,
# #                                     score_cutoff=score_cutoff)
# #         if result:
# #             df_inspect.loc[idx_insp, 'fuzzy_matched'] = True
# #             df_rest.loc[idx_rest, 'fuzzy_matched'] = True
# #             df_rest.loc[idx_rest, 'matched_address'] = row_insp['address']
# #             df_rest.loc[idx_rest, 'matched_name'] = row_insp['name']
# #             df_rest.loc[idx_rest, 'matched_index'] = idx_insp
# #             # print('=' * 30)
# #             # print(idx_insp)
# #             # print(result)
# #             break